In [57]:
year = 2025; day = 10

from aocd import get_data, submit
import numpy as np
data = get_data(year=year, day=day)
# data = """[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
# [...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
# [.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}"""

data=data.split("\n")
data = [d.split() for d in data]
# data = [list(map(int, line.split(","))) for line in data]
# data = np.array(data)

def convert_button(but):
    new_but = [0]*10
    for i in but:
        new_but[i] = 1
    return tuple(new_but)

def parse_line(line):
    target = line[0][1:-1]
    target = tuple([1 if c == "#" else 0 for c in target])
    joltages = tuple(map(int, line[-1][1:-1].split(",")))
    buts = line[1:-1]
    buts = [tuple(map(int, b[1:-1].split(","))) for b in buts]
    buts = [convert_button(but)[:len(target)] for but in buts]
    return target, buts, joltages
data = [parse_line(line) for line in data]
data

[((1, 1, 1, 0),
  [(1, 0, 0, 1), (1, 0, 1, 1), (0, 1, 0, 1), (1, 1, 0, 1)],
  (117, 15, 2, 128)),
 ((0, 0, 0, 1, 1, 0, 0),
  [(0, 0, 1, 1, 0, 0, 0),
   (0, 0, 0, 0, 0, 1, 1),
   (1, 1, 0, 1, 1, 1, 0),
   (1, 1, 1, 0, 0, 0, 1),
   (0, 0, 1, 1, 1, 1, 1),
   (1, 1, 0, 0, 0, 1, 0)],
  (12, 12, 13, 13, 2, 31, 19)),
 ((0, 1, 1, 1, 0, 1, 0, 0, 1, 0),
  [(1, 1, 0, 1, 0, 0, 0, 1, 1, 0),
   (1, 1, 1, 1, 1, 0, 1, 1, 0, 1),
   (1, 1, 0, 1, 0, 1, 1, 0, 1, 1),
   (0, 0, 0, 1, 0, 0, 0, 0, 1, 0),
   (0, 1, 0, 0, 0, 0, 1, 1, 0, 1),
   (0, 0, 1, 0, 0, 0, 0, 0, 1, 0),
   (0, 1, 0, 1, 0, 1, 0, 0, 0, 0),
   (1, 0, 0, 0, 0, 0, 1, 0, 0, 1),
   (0, 0, 0, 0, 1, 0, 0, 0, 1, 0),
   (1, 1, 0, 1, 0, 0, 1, 1, 1, 1),
   (1, 0, 1, 0, 0, 1, 1, 0, 1, 0)],
  (51, 33, 24, 38, 16, 22, 58, 25, 54, 44)),
 ((1, 0, 1, 1), [(1, 1, 1, 1), (1, 0, 1, 1)], (9, 1, 9, 9)),
 ((1, 1, 0, 1, 1, 1),
  [(1, 1, 0, 0, 1, 1),
   (1, 1, 0, 1, 0, 1),
   (1, 1, 0, 0, 1, 0),
   (0, 0, 1, 1, 0, 0),
   (1, 1, 0, 1, 1, 1),
   (0, 1, 0, 1, 0, 0)],
 

In [39]:
def add(a, b):
    return tuple((i+j) % 2 for i, j in zip(a, b))

In [40]:
len(data[0])

3

In [41]:
presses = []
for line in data:
    target, buttons, joltages = line

    seen_states = set()
    current_states = [[0]*len(target)]
    for iteration in range(1_000):
        # print(iteration)

        new_to_visit = set()
        is_done = False
        for s in current_states:
            for buts in buttons:
                new_s = add(s, buts)
                if new_s == target:
                    is_done = True
                    presses.append(iteration+1)
                    break
                if new_s not in seen_states:
                    new_to_visit.add(new_s)
                    seen_states.add(new_s)
            if is_done: break
        if is_done:
            break
        current_states = list(new_to_visit)

    else:
        raise ValueError("max iterations exceeded")

In [42]:
sum(presses)

7

In [43]:
target, buttons, joltages = data[0]

A = np.array(buttons)
b = np.array(joltages)

Ap = np.linalg.pinv(A)
x = np.dot(Ap.T, b).astype("int")
tuple(map(int, np.dot(A.T, x)))
int(sum(x))

8

In [44]:
np.array([0., 0.4, 0.5, 0.6]).astype("int")

array([0, 0, 0, 0])

In [45]:
x

array([1, 3, 0, 2, 1, 1])

In [46]:
U, S, Vh = np.linalg.svd(A, full_matrices=True)
Vh

array([[ 0.37174803,  0.37174803,  0.60150096,  0.60150096],
       [-0.37174803,  0.37174803, -0.60150096,  0.60150096],
       [-0.60150096, -0.60150096,  0.37174803,  0.37174803],
       [-0.60150096,  0.60150096,  0.37174803, -0.37174803]])

In [47]:
len(data)

3

In [48]:

def add_num(a, b):
    return tuple((i+j) for i, j in zip(a, b))

def overshot(state, target):
    return any(s > t for t, s in zip(target, state))

presses = []
# for idx, line in enumerate(data):

# print(idx)
target, buttons, joltages = line
target, buttons, joltages = data[14]

A = np.array(buttons)
b = np.array(joltages)

Ap = np.linalg.pinv(A)
x = np.dot(Ap.T, b).astype("int")
x = x - 5*np.ones(len(x))
x = np.clip(x, 0, np.inf)
start = tuple(map(int, np.dot(A.T, x)))
presses_offset = int(sum(x))

seen_states = set()


# current_states = [start]
current_states = [[0]*len(target)]

print(f"{presses_offset=} {joltages=} {start=}")

for iteration in range(1_000):
    print(len(current_states))

    new_to_visit = set()
    is_done = False
    # print(current_states)
    for s in current_states:
        for buts in buttons:
            new_s = add_num(s, buts)
            if new_s == joltages:
                is_done = True
                presses.append(iteration+1)
                break
            if new_s not in seen_states and not overshot(new_s, joltages):
                new_to_visit.add(new_s)
                seen_states.add(new_s)
        if is_done: break
    if is_done:
        p = iteration+1+presses_offset
        print(p)
        presses.append(p)
        break
    current_states = list(new_to_visit)

else:
    raise ValueError("max iterations exceeded")

IndexError: list index out of range

In [12]:
np.dot(Ap.T, b)

array([ 31.63636364,  23.81818182,   1.18181818,  30.81818182,
         2.18181818, 136.18181818,  19.        ,  17.81818182,
        -3.81818182])

In [ ]:
%pip install pulp


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [33]:
buttons

[(0, 1, 0, 1, 1, 1, 0, 1),
 (0, 0, 1, 0, 0, 0, 1, 0),
 (1, 1, 0, 1, 1, 0, 0, 1),
 (1, 1, 0, 0, 1, 0, 0, 1),
 (0, 1, 1, 0, 1, 1, 0, 1),
 (0, 1, 0, 1, 0, 1, 0, 1),
 (1, 0, 1, 0, 0, 0, 0, 0),
 (1, 1, 1, 1, 0, 1, 0, 0),
 (1, 1, 1, 1, 1, 1, 1, 0)]

In [34]:
joltages

(65, 216, 59, 183, 62, 184, 20, 202)

In [37]:
buttons

[(0, 1, 0, 0, 0, 0, 0, 1),
 (0, 0, 1, 1, 0, 0, 0, 0),
 (1, 0, 1, 1, 1, 1, 1, 1),
 (0, 1, 0, 0, 1, 1, 0, 1),
 (1, 1, 1, 0, 1, 1, 0, 0),
 (1, 0, 0, 1, 0, 0, 1, 0)]

In [51]:
buttons

[(0, 0, 0, 1),
 (0, 1, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 1, 1),
 (1, 0, 1, 0),
 (1, 1, 0, 0)]

In [ ]:
import pulp
presses = 0
for line in data:
    target, buttons, joltages = line

    A = np.array(buttons).T
    b = np.array(joltages)
    # A = [[1, 2, 1], [3, 1, -1]]
    # b = [20, 12]
    mod = pulp.LpProblem('prob', pulp.LpMinimize)
    vars = pulp.LpVariable.dicts('x', range(len(A[0])), lowBound=0, cat='Integer')

    # Add constraints
    for row, joltage in zip(A, b):
        mod += sum([row[i]*vars[i] for i in range(len(row))]) == joltage

    # Minimize sum of variables
    mod += pulp.lpSum([vars[i] for i in range(len(A[0]))])

    status = mod.solve()
    print(f"Status: {pulp.LpStatus[status]}")

    if status == 1:  # Optimal solution found
        solution = [int(vars[i].value()) for i in range(len(A[0]))]
        print(f"Solution: {solution}")
        print(f"Sum: {sum(solution)}")
        presses += sum(solution)
    else:
        print("No solution found")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/rikvandervlist/git/personal/adventofcode/.venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/35/6jcttdh11q3cjlxc8qcds6rw0000gn/T/6243ffe2f7b14b91b035e49c5b0f72ad-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/35/6jcttdh11q3cjlxc8qcds6rw0000gn/T/6243ffe2f7b14b91b035e49c5b0f72ad-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 32 RHS
At line 37 BOUNDS
At line 42 ENDATA
Problem MODEL has 4 rows, 4 columns and 10 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 128 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 128 to -1.79769e+308
Probing was tried 0 times an

In [60]:
presses

16386

In [54]:
np.dot(A, np.array(solution))

array([3, 5, 4, 7])

In [27]:
buttons

[(0, 1, 0, 1, 1, 1, 0, 1),
 (0, 0, 1, 0, 0, 0, 1, 0),
 (1, 1, 0, 1, 1, 0, 0, 1),
 (1, 1, 0, 0, 1, 0, 0, 1),
 (0, 1, 1, 0, 1, 1, 0, 1),
 (0, 1, 0, 1, 0, 1, 0, 1),
 (1, 0, 1, 0, 0, 0, 0, 0),
 (1, 1, 1, 1, 0, 1, 0, 0),
 (1, 1, 1, 1, 1, 1, 1, 0)]

In [28]:
joltages

(65, 216, 59, 183, 62, 184, 20, 202)

In [13]:
%pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.1/37.1 MB 12.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from z3 import *

target, buttons, joltages = data[14]

A = np.array(buttons).T
b = joltages

n = len(A[0]) # number of variables
m = len(b)    # number of constraints

X = [ Int('x%d' % i) for i in range(n) ]
s = Optimize() 
s.add(And([ X[i] >= 0 for i in range(n) ]))
for i in range(m):
    s.add( Sum([ A[i][j]*X[j] for j in range(n) ]) == b[i])

# Minimize the sum of X
s.minimize(Sum(X))

if s.check() == sat:
    sol = [s.model().evaluate(X[i]) for i in range(n)]
    total = sum([sol[i].as_long() for i in range(n)])
else:
    raise ValueError(f"No solution found {s=}")